In [190]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import numpy as np


In [163]:
df = pd.read_csv("../data/dataset.csv")
df.describe()

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,Year,Month,tsunami
count,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000,782.000000
mean,6.941125,4.333760,5.964194,870.108696,230.250639,1.325757,25.038990,75.883199,3.538100,52.609199,2012.280051,6.563939,0.388747
std,0.445514,3.169939,1.462724,322.465367,250.188177,2.218805,24.225067,137.277078,27.303429,117.898886,6.099439,3.507866,0.487778
min,6.500000,0.000000,1.000000,650.000000,0.000000,0.000000,0.000000,2.700000,-61.848400,-179.968000,2001.000000,1.000000,0.000000
25%,6.600000,0.000000,5.000000,691.000000,0.000000,0.000000,14.625000,14.000000,-14.595600,-71.668050,2007.000000,3.250000,0.000000
50%,6.800000,5.000000,6.000000,754.000000,140.000000,0.000000,20.000000,26.295000,-2.572500,109.426000,2013.000000,7.000000,0.000000
75%,7.100000,7.000000,7.000000,909.750000,445.000000,1.863000,30.000000,49.750000,24.654500,148.941000,2017.000000,10.000000,1.000000
max,9.100000,9.000000,9.000000,2910.000000,934.000000,17.654000,239.000000,670.810000,71.631200,179.662000,2022.000000,12.000000,1.000000


In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   magnitude  782 non-null    float64
 1   cdi        782 non-null    int64  
 2   mmi        782 non-null    int64  
 3   sig        782 non-null    int64  
 4   nst        782 non-null    int64  
 5   dmin       782 non-null    float64
 6   gap        782 non-null    float64
 7   depth      782 non-null    float64
 8   latitude   782 non-null    float64
 9   longitude  782 non-null    float64
 10  Year       782 non-null    int64  
 11  Month      782 non-null    int64  
 12  tsunami    782 non-null    int64  
dtypes: float64(6), int64(7)
memory usage: 79.6 KB


In [165]:
df

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,Year,Month,tsunami
0,7.0,8,7,768,117,0.509,17.0,14.000,-9.7963,159.596,2022,11,1
1,6.9,4,4,735,99,2.229,34.0,25.000,-4.9559,100.738,2022,11,0
2,7.0,3,3,755,147,3.125,18.0,579.000,-20.0508,-178.346,2022,11,1
3,7.3,5,5,833,149,1.865,21.0,37.000,-19.2918,-172.129,2022,11,1
4,6.6,0,2,670,131,4.998,27.0,624.464,-25.5948,178.278,2022,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,7.7,0,8,912,427,0.000,0.0,60.000,13.0490,-88.660,2001,1,0
778,6.9,5,7,745,0,0.000,0.0,36.400,56.7744,-153.281,2001,1,0
779,7.1,0,7,776,372,0.000,0.0,103.000,-14.9280,167.170,2001,1,0
780,6.8,0,5,711,64,0.000,0.0,33.000,6.6310,126.899,2001,1,0


In [166]:
q1 = df.quantile(0.25)
q3 = df.quantile(0.75)
q = q3 - q1
out = ((df < (q1 - 1.5 * q))) | (df > (q3 + 1.5 * q))
mask = ~((df < (q1 - 1.5 * q)) | (df > (q3 + 1.5 * q))).any(axis=1)

df_clean = df[mask]
df_clean

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,Year,Month,tsunami
0,7.0,8,7,768,117,0.509,17.0,14.0,-9.7963,159.5960,2022,11,1
1,6.9,4,4,735,99,2.229,34.0,25.0,-4.9559,100.7380,2022,11,0
3,7.3,5,5,833,149,1.865,21.0,37.0,-19.2918,-172.1290,2022,11,1
7,6.7,7,6,797,145,1.151,37.0,20.0,7.6712,-82.3396,2022,10,1
10,6.9,9,9,887,215,0.401,34.0,10.0,23.1444,121.3070,2022,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,7.7,0,8,912,427,0.000,0.0,60.0,13.0490,-88.6600,2001,1,0
778,6.9,5,7,745,0,0.000,0.0,36.4,56.7744,-153.2810,2001,1,0
779,7.1,0,7,776,372,0.000,0.0,103.0,-14.9280,167.1700,2001,1,0
780,6.8,0,5,711,64,0.000,0.0,33.0,6.6310,126.8990,2001,1,0


In [167]:
df_clean.corr()

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,Year,Month,tsunami
magnitude,1.000000,0.080443,0.203253,0.638710,0.088381,-0.056585,-0.136090,0.076492,-0.092118,0.049241,-0.058653,-0.021944,-0.025515
cdi,0.080443,1.000000,0.123543,0.445454,-0.270030,0.201054,0.221652,0.030442,0.080437,-0.154117,0.544292,0.075229,0.249882
mmi,0.203253,0.123543,1.000000,0.318975,0.125806,-0.188805,-0.050271,-0.024938,0.084430,-0.051808,-0.180176,-0.052974,-0.140313
sig,0.638710,0.445454,0.318975,1.000000,-0.034174,-0.007187,0.006263,0.103766,0.098166,-0.115432,0.163169,0.010091,0.030965
nst,0.088381,-0.270030,0.125806,-0.034174,1.000000,-0.665137,-0.095620,0.001661,0.136825,0.140147,-0.688343,-0.056027,-0.643587
dmin,-0.056585,0.201054,-0.188805,-0.007187,-0.665137,1.000000,0.067430,-0.019393,-0.022782,-0.003194,0.614582,0.018877,0.578867
gap,-0.136090,0.221652,-0.050271,0.006263,-0.095620,0.067430,1.000000,-0.117164,-0.086891,-0.286410,0.257561,0.122469,0.108087
depth,0.076492,0.030442,-0.024938,0.103766,0.001661,-0.019393,-0.117164,1.000000,-0.015275,-0.021497,-0.014099,-0.033116,0.023628
latitude,-0.092118,0.080437,0.084430,0.098166,0.136825,-0.022782,-0.086891,-0.015275,1.000000,0.019168,-0.026077,-0.000601,-0.059791
longitude,0.049241,-0.154117,-0.051808,-0.115432,0.140147,-0.003194,-0.286410,-0.021497,0.019168,1.000000,-0.094088,-0.008539,-0.120691


In [168]:
Y = df_clean['tsunami']
X = df_clean.drop('tsunami',axis=1)
X

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,Year,Month
0,7.0,8,7,768,117,0.509,17.0,14.0,-9.7963,159.5960,2022,11
1,6.9,4,4,735,99,2.229,34.0,25.0,-4.9559,100.7380,2022,11
3,7.3,5,5,833,149,1.865,21.0,37.0,-19.2918,-172.1290,2022,11
7,6.7,7,6,797,145,1.151,37.0,20.0,7.6712,-82.3396,2022,10
10,6.9,9,9,887,215,0.401,34.0,10.0,23.1444,121.3070,2022,9
...,...,...,...,...,...,...,...,...,...,...,...,...
777,7.7,0,8,912,427,0.000,0.0,60.0,13.0490,-88.6600,2001,1
778,6.9,5,7,745,0,0.000,0.0,36.4,56.7744,-153.2810,2001,1
779,7.1,0,7,776,372,0.000,0.0,103.0,-14.9280,167.1700,2001,1
780,6.8,0,5,711,64,0.000,0.0,33.0,6.6310,126.8990,2001,1


In [169]:
model = LogisticRegression(fit_intercept=False,random_state=21,max_iter=2000)
model.fit(X,Y)
y_pred = model.predict(X)
accir = accuracy_score(Y,y_pred=y_pred)
accir


0.8598326359832636

In [170]:
mode = [df['tsunami'].mode()] * len(Y)
accur2 = accuracy_score(Y,mode)
accur2

0.6652719665271967

In [174]:
df_clean.loc[:,'pred_log'] = y_pred
df_clean

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,Year,Month,tsunami,pred_log
0,7.0,8,7,768,117,0.509,17.0,14.0,-9.7963,159.5960,2022,11,1,0
1,6.9,4,4,735,99,2.229,34.0,25.0,-4.9559,100.7380,2022,11,0,1
3,7.3,5,5,833,149,1.865,21.0,37.0,-19.2918,-172.1290,2022,11,1,1
7,6.7,7,6,797,145,1.151,37.0,20.0,7.6712,-82.3396,2022,10,1,0
10,6.9,9,9,887,215,0.401,34.0,10.0,23.1444,121.3070,2022,9,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,7.7,0,8,912,427,0.000,0.0,60.0,13.0490,-88.6600,2001,1,0,0
778,6.9,5,7,745,0,0.000,0.0,36.4,56.7744,-153.2810,2001,1,0,1
779,7.1,0,7,776,372,0.000,0.0,103.0,-14.9280,167.1700,2001,1,0,0
780,6.8,0,5,711,64,0.000,0.0,33.0,6.6310,126.8990,2001,1,0,1


In [175]:
scaler = StandardScaler()
x_name = X.columns
x_name = list(x_name)
df_scaled = df_clean.copy()
df_scaled[x_name] = scaler.fit_transform(df_clean[x_name])
df_scaled

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,Year,Month,tsunami,pred_log
0,0.402862,1.316053,0.747840,-0.077780,-0.606420,-0.183475,-0.332138,-0.704942,-0.523982,0.803822,1.859892,1.251790,1,0
1,0.106603,0.025646,-1.906623,-0.343057,-0.681101,1.274623,1.068679,-0.096986,-0.332843,0.251284,1.859892,1.251790,0,1
3,1.291637,0.348248,-1.021802,0.444735,-0.473655,0.966049,-0.002534,0.566238,-0.898943,-2.310296,1.859892,1.251790,1,1
7,-0.485913,0.993451,-0.136981,0.155342,-0.490250,0.360768,1.315882,-0.373330,0.165780,-1.467385,1.859892,0.969812,1,0
10,0.106603,1.638655,2.517483,0.878825,-0.199825,-0.275030,1.068679,-0.926017,0.776790,0.444378,1.859892,0.687835,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,2.476671,-1.264761,1.632662,1.079792,0.679747,-0.614971,-1.732955,1.837418,0.378140,-1.526718,-1.652071,-1.567981,0,0
778,0.106603,0.348248,0.747840,-0.262670,-1.091845,-0.614971,-1.732955,0.533077,2.104781,-2.133358,-1.652071,-1.567981,0,1
779,0.699120,-1.264761,0.747840,-0.013471,0.451556,-0.614971,-1.732955,4.213972,-0.726624,0.874924,-1.652071,-1.567981,0,0
780,-0.189655,-1.264761,-1.021802,-0.535986,-0.826313,-0.614971,-1.732955,0.345163,0.124704,0.496874,-1.652071,-1.567981,0,1


In [177]:
model2 = LogisticRegression(max_iter=2000,fit_intercept=False,random_state=21)
X_sc = df_clean.drop(['tsunami','pred_log'],axis=1)
model2.fit(X_sc,df_clean['tsunami'])



LogisticRegression(fit_intercept=False, max_iter=2000, random_state=21)

In [178]:
y_p = model2.predict(X_sc)
new_acc = accuracy_score(df_clean['tsunami'],y_p)
new_acc


0.8598326359832636

In [185]:
model_svc=SVC(kernel='linear',probability=True, random_state=21)
model_svc.fit(X_sc,df_clean['tsunami'])
pred_y=model_svc.predict(X_sc)
acc2 = accuracy_score(pred_y,df_clean['tsunami'])
df_clean.loc[:,'pred_svc'] = pred_y


In [186]:
df_clean

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,Year,Month,tsunami,pred_log,pred_svc
0,7.0,8,7,768,117,0.509,17.0,14.0,-9.7963,159.5960,2022,11,1,0,1
1,6.9,4,4,735,99,2.229,34.0,25.0,-4.9559,100.7380,2022,11,0,1,1
3,7.3,5,5,833,149,1.865,21.0,37.0,-19.2918,-172.1290,2022,11,1,1,1
7,6.7,7,6,797,145,1.151,37.0,20.0,7.6712,-82.3396,2022,10,1,0,1
10,6.9,9,9,887,215,0.401,34.0,10.0,23.1444,121.3070,2022,9,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,7.7,0,8,912,427,0.000,0.0,60.0,13.0490,-88.6600,2001,1,0,0,0
778,6.9,5,7,745,0,0.000,0.0,36.4,56.7744,-153.2810,2001,1,0,1,0
779,7.1,0,7,776,372,0.000,0.0,103.0,-14.9280,167.1700,2001,1,0,0,0
780,6.8,0,5,711,64,0.000,0.0,33.0,6.6310,126.8990,2001,1,0,1,0


In [188]:
model_tree = DecisionTreeClassifier(max_depth=4,random_state=42)
model_tree.fit(X_sc,df_clean['tsunami'])
pr_y = model_tree.predict(X_sc)
accur_tree = accuracy_score(df_clean['tsunami'],pr_y)
df_clean.loc[:,'pred_desion'] = pr_y
df_clean

,magnitude,cdi,mmi,sig,nst,dmin,gap,depth,latitude,longitude,Year,Month,tsunami,pred_log,pred_svc,pred_desion
0,7.0,8,7,768,117,0.509,17.0,14.0,-9.7963,159.5960,2022,11,1,0,1,1
1,6.9,4,4,735,99,2.229,34.0,25.0,-4.9559,100.7380,2022,11,0,1,1,0
3,7.3,5,5,833,149,1.865,21.0,37.0,-19.2918,-172.1290,2022,11,1,1,1,1
7,6.7,7,6,797,145,1.151,37.0,20.0,7.6712,-82.3396,2022,10,1,0,1,1
10,6.9,9,9,887,215,0.401,34.0,10.0,23.1444,121.3070,2022,9,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,7.7,0,8,912,427,0.000,0.0,60.0,13.0490,-88.6600,2001,1,0,0,0,0
778,6.9,5,7,745,0,0.000,0.0,36.4,56.7744,-153.2810,2001,1,0,1,0,0
779,7.1,0,7,776,372,0.000,0.0,103.0,-14.9280,167.1700,2001,1,0,0,0,0
780,6.8,0,5,711,64,0.000,0.0,33.0,6.6310,126.8990,2001,1,0,1,0,0


In [200]:
df_clean.loc[:, 'final_point'] = np.where(
    (df_clean['pred_log'] + df_clean['pred_svc'] + df_clean['pred_desion']) > 1,
    df_clean['pred_log'] + df_clean['pred_svc'] + df_clean['pred_desion'],
    0
)
df_clean.loc[:, 'new_column'] = np.where(
    df_clean['final_point'] > 1,
    1,
    0
)
df_clean
final_accuracy = accuracy_score(df_clean['tsunami'],df_clean['new_column'])
final_accuracy

0.8828451882845189